#### Content of the notebook
1. length of conversations 
2. no. of users 

In [1]:
import pandas as pd
import json
import pprint

In [2]:
path = 'data/dbpedia_chats.json'

with open(path, 'r') as file:
    data = json.load(file)
    flattened_rows = []
    
    for row in data['rows']:
        flattened_rows.append(row['doc'])
    
    df = pd.DataFrame(flattened_rows) 
    print(len(flattened_rows))
        
    pprint.pprint(data['rows'][:2])
#     print(data['rows'][0]['doc'].keys())
#     print(data.keys())


90980
[{'doc': {'_id': '000067a205ca41daa55366175261ddb7',
          '_rev': '1-7fba37dd9b65dac2545549bbf93e23f6',
          'fromBot': True,
          'id': 'dbfaba55-ecc0-42a3-b2c7-1daa39fb36b3_1553869482210_response',
          'response': [{'messageData': [{'buttons': [],
                                         'fields': [],
                                         'image': 'http://i.imgur.com/044oUKK.png',
                                         'smartReplies': [],
                                         'text': "I'm not sure I understand "
                                                 'you fully.'}],
                        'messageType': 'text'},
                       {'messageData': [{'buttons': [],
                                         'fields': [],
                                         'image': 'http://i.imgur.com/044oUKK.png',
                                         'smartReplies': [{'slackStyle': 'primary',
                                                     

In [3]:
df.drop(columns=['_id', '_rev', 'language', 'views'], inplace=True)
df.head()

,id,userId,fromBot,response,timestamp,request
0,dbfaba55-ecc0-42a3-b2c7-1daa39fb36b3_155386948...,dbfaba55-ecc0-42a3-b2c7-1daa39fb36b3,True,"[{'messageType': 'text', 'messageData': [{'ima...",1.553869e+12,NaN
1,448e21ab-e862-42d3-b106-50608fea9381_151598711...,448e21ab-e862-42d3-b106-50608fea9381,False,NaN,1.515987e+12,{'userId': '448e21ab-e862-42d3-b106-50608fea93...
2,cded1ee9-9196-43cc-810e-fc9b09b35ed6_152268317...,cded1ee9-9196-43cc-810e-fc9b09b35ed6,False,NaN,1.522683e+12,{'userId': 'cded1ee9-9196-43cc-810e-fc9b09b35e...
3,03b12284-8cc5-47e2-9786-2319ee6b32ea_155369517...,03b12284-8cc5-47e2-9786-2319ee6b32ea,False,NaN,1.553695e+12,{'userId': '03b12284-8cc5-47e2-9786-2319ee6b32...
4,xqmzefkegm_1530790477188_response,xqmzefkegm,True,"[{'messageType': 'text', 'messageData': [{'ima...",1.530790e+12,NaN


In [5]:
# print(any(df['userId'].duplicated()))
## length of conversations

df.groupby(['userId']).count()

,id,fromBot,response,timestamp,request
userId,,,,,
00060d94-a6e3-4fc9-b506-d75ab5448a48,20,20,10,20,10
0011a66e-5d54-4fdb-adb1-8cf86b31895b,44,44,22,44,22
001c5c03-9f33-43a2-bdea-2c2505c0c19b,6,6,3,6,3
003c031c-3041-4f88-aa96-253a1b75b392,2,2,1,2,1
004d59be-ddca-490c-9ba9-b468c4505392,6,6,3,6,3
00645c9e-fdc7-4067-8811-3d3d9c4da089,10,10,5,10,5
006b78e3-54b5-4c44-99f0-446359c552cf,6,6,3,6,3
0072762f-e180-4a05-9f7b-ca83f5dd8dd5,12,12,6,12,6
007354e2-06d2-47ff-8e01-548a8870991f,12,12,6,12,6


In [5]:
df['userId'].isnull().values.sum()

2

In [14]:
# new data format 
# grouped = df.groupby(['userId'], as_index=False)
# grouped.head(10)

In [4]:
# combine all chats with the same userId 

from collections import defaultdict
grouped_chats = defaultdict(list)
for i, row in df.iterrows():
    conv = {}
    conv['id'] = row['id']
    conv['fromBot'] = row['fromBot']
    conv['request'] = row['request']
    conv['response'] = row['response']
    conv['timestamp'] = row['timestamp']
    conv['userId'] = row['userId']
    grouped_chats[row['userId']].append(conv) 
print(len(grouped_chats))


9084


In [5]:
# sort each conversation based on time and get the length of conversation per user
import operator 

conv_len = dict()

for k, v in grouped_chats.items(): # {userId: [dict1, dict2, ]}
    # v - a list of dicts ; sort in-place based on timestamp
    v.sort(key=operator.itemgetter('timestamp'))
    conv_len[k] = len(v)

In [11]:
# get the avg length of conversation
from statistics import mean 
from collections import Counter
print(mean(list(conv_len.values())))
print(max(list(conv_len.values())))
print(min(list(conv_len.values())))
print(conv_len.values())

freq = Counter()
for k, v in conv_len.items():
    freq[v] += 1
    
print(freq)


10.015411712901805
1661
1
dict_values([24, 62, 10, 72, 2, 96, 4, 12, 26, 18, 22, 22, 16, 14, 16, 69, 46, 32, 10, 6, 12, 12, 2, 74, 280, 2, 2, 38, 16, 2, 6, 9, 22, 22, 12, 4, 42, 43, 14, 2, 2, 6, 26, 47, 28, 2, 30, 8, 20, 44, 18, 24, 10, 381, 68, 26, 12, 40, 54, 16, 24, 32, 62, 24, 2, 30, 36, 26, 2, 28, 2, 18, 32, 14, 51, 2, 2, 34, 40, 32, 10, 219, 98, 16, 10, 218, 22, 18, 12, 60, 22, 598, 16, 22, 36, 47, 2, 186, 319, 2, 148, 16, 32, 28, 2, 51, 26, 2, 127, 18, 80, 6, 38, 28, 10, 30, 18, 16, 30, 68, 2, 2, 232, 2, 20, 6, 6, 20, 16, 18, 38, 2, 10, 20, 208, 2, 2, 16, 22, 8, 58, 8, 26, 4, 2, 1661, 26, 40, 2, 175, 40, 24, 36, 44, 8, 20, 18, 37, 12, 40, 4, 14, 26, 20, 13, 314, 56, 10, 20, 6, 16, 44, 66, 50, 34, 20, 28, 2, 36, 111, 16, 11, 2, 26, 12, 24, 18, 18, 16, 45, 14, 40, 126, 10, 6, 142, 38, 14, 56, 14, 14, 8, 16, 56, 6, 40, 10, 26, 22, 12, 16, 2, 40, 2, 52, 72, 2, 28, 44, 47, 20, 20, 34, 10, 28, 20, 4, 8, 10, 18, 12, 14, 15, 78, 62, 8, 92, 24, 24, 61, 14, 40, 20, 94, 32, 19, 16, 24, 24,

In [14]:
with open('data/grouped_chats.json', 'w') as outfile:
    json.dump(grouped_chats, outfile)

In [13]:
# new_df = df.groupby(['userId'], as_index=False).agg(lambda x: x.tolist())
# print(new_df.shape)
# print(any(new_df.index.duplicated()))

In [45]:
# new_df.to_csv('data/combined_user_chats.csv')

In [47]:
# new_path = 'data/combined_user_chats.csv'
# df = pd.read_csv(new_path)

In [12]:
# df['request'][0]

In [11]:
# df['response'][0]